# Imports

In [1]:
import os
import numpy as np

import requests
import pandas as pd
from pandas import json_normalize

In [6]:
os.getcwd()
os.chdir('..')

In [19]:
AQOPEN_API_KEY = '4f3f6e2b1360980aec87f727b4e6d7fb70a6169f'

cities = ['los_angeles', 'boston', 'san_francisco', 'shanghai',
           'seoul', 'paris', 'london']

# Merge Historical Data

In [ ]:
# create a single csv w/ all historical data
# create functions to add it to feature store
# make functions for real time data to go in a script?
# create tests?

In [ ]:
merged_data = pd.DataFrame()
for f in os.listdir('data/air_quality'):
    if f.endswith('.csv'):
        city_name = f.replace('_', ' ').split('.')[0]
        # print(city_name)
        df = pd.read_csv(f'data/air_quality/{f}')
        df['city'] = city_name  # Add a new column for the city name
        cols = ['date', 'city'] + [col for col in df.columns if col not in ['date', 'city']]
        df = df[cols]
        merged_data = pd.concat([merged_data, df], ignore_index=True)


merged_data.to_csv('data/air_quality/merged_data.csv', index=False)

boston
london
los angeles
paris
san francisco
seoul
shanghai


# Add Historical Data to Feature Store

In [14]:
get_air_json("london", AQOPEN_API_KEY)

{'aqi': 53,
 'idx': 5724,
 'attributions': [{'url': 'http://uk-air.defra.gov.uk/',
   'name': 'UK-AIR, air quality information resource - Defra, UK',
   'logo': 'UK-Department-for-environment-food-and-rural-affairs.png'},
  {'url': 'https://londonair.org.uk/',
   'name': "London Air Quality Network - Environmental Research Group, King's College London",
   'logo': 'UK-London-Kings-College.png'},
  {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
 'city': {'geo': [51.5073509, -0.1277583],
  'name': 'London',
  'url': 'https://aqicn.org/city/london',
  'location': ''},
 'dominentpol': 'pm25',
 'iaqi': {'co': {'v': 4.6},
  'h': {'v': 38.5},
  'no2': {'v': 15.6},
  'o3': {'v': 16.6},
  'p': {'v': 1024.5},
  'pm10': {'v': 26},
  'pm25': {'v': 53},
  'so2': {'v': 1.1},
  't': {'v': 9.5},
  'w': {'v': 10}},
 'time': {'s': '2025-03-18 12:00:00',
  'tz': '+00:00',
  'v': 1742299200,
  'iso': '2025-03-18T12:00:00Z'},
 'forecast': {'daily': {'o3': [{'avg': 10,
     'day'

# Add New Data to Feature Store

In [13]:
def get_air_json(city_name, AIR_QUALITY_API_KEY):
    return requests.get(f'https://api.waqi.info/feed/{city_name}/?token={AQOPEN_API_KEY}').json()['data']

In [16]:
def get_air_quality_data(city_name):
    # AIR_QUALITY_API_KEY = os.getenv('AIR_QUALITY_API_KEY')
    json = get_air_json(city_name, AQOPEN_API_KEY)
    print(json)
    iaqi = json['iaqi']
    forecast = json['forecast']['daily']
    
    params = ['pm10', 'pm25', 'no2', 'so2', 'co', 'o3']
    for param in params:
        if param not in iaqi: # if the parameter is not present in the data, add a column with NaN values
            iaqi[param] = {"v": np.nan}
    return [
        city_name,
        json['aqi'],                 # AQI
        json['time']['s'][:10],      # Date
        iaqi['pm25']['v'],
        iaqi['pm10']['v'],
        iaqi['no2']['v'],
        iaqi['so2']['v'],
        iaqi['co']['v'],
        iaqi['o3']['v']
    ]

In [18]:
get_air_quality_data('istanbul')#, AQOPEN_API_KEY)

{'aqi': 88, 'idx': 4143, 'attributions': [{'url': 'https://sim.csb.gov.tr/Services/AirQuality', 'name': 'Turkey National Air Quality Monitoring Network (Ulusal Hava Kalitesi İzleme Ağı)', 'logo': 'Turkey-TC-cevre-ve-sehircilik-bakanligi.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [41.014722, 28.954722], 'name': 'Istanbul (Aksaray), Turkey', 'url': 'https://aqicn.org/city/turkey/marmara/istanbul/aksaray', 'location': ''}, 'dominentpol': 'pm25', 'iaqi': {'co': {'v': 4.5}, 'dew': {'v': 5}, 'h': {'v': 51}, 'no2': {'v': 23.7}, 'o3': {'v': 5.2}, 'p': {'v': 1013}, 'pm10': {'v': 64}, 'pm25': {'v': 88}, 'so2': {'v': 2.1}, 't': {'v': 15}, 'w': {'v': 2.3}}, 'time': {'s': '2025-03-14 05:00:00', 'tz': '+03:00', 'v': 1741928400, 'iso': '2025-03-14T05:00:00+03:00'}, 'forecast': {'daily': {'o3': [{'avg': 10, 'day': '2025-03-12', 'max': 18, 'min': 6}, {'avg': 5, 'day': '2025-03-13', 'max': 11, 'min': 1}, {'avg': 8, 'day': '2025-03-14', 'max': 19, 'm

['istanbul', 88, '2025-03-14', 88, 64, 23.7, 2.1, 4.5, 5.2]

In [69]:
from datetime import datetime


def get_air_quality_df(data):
    col_names = [
        'city',
        'aqi',
        'date',
        'pm25',
        'pm10',
        'no2',
        'so2',
        'co',
        'o3'
    ]

    new_data = pd.DataFrame(
        data,
        columns=col_names
    )
    new_data.date = new_data.date.apply(timestamp_2_time)

    return new_data

def timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

In [70]:
df = get_air_quality_df([get_air_quality_data(city) for city in cities])

In [71]:
df

,city,aqi,date,pm25,pm10,no2,so2,co,o3
0,Brooklyn,53,1741924800000,53,NaN,NaN,NaN,NaN,NaN
1,Honolulu,37,1741924800000,37,24.0,NaN,0.1,0.9,NaN
2,Boston,31,1741924800000,31,19.0,2.8,0.3,3.4,28.1
3,Nashville,51,1741924800000,51,35.0,NaN,NaN,NaN,NaN
4,Seattle,13,1741924800000,13,NaN,12.9,NaN,NaN,NaN
5,Albuquerque,84,1741924800000,26,84.0,4.7,NaN,1.2,32.8
6,Denver,17,1741924800000,13,10.0,4.6,0.6,3.2,16.7
7,Miami,45,1741924800000,45,31.0,NaN,0.3,2.8,13.0
8,Austin,65,1741924800000,65,NaN,2.2,1.3,NaN,24.1
9,Atlanta,36,1741924800000,36,NaN,NaN,1.0,NaN,28.9


In [10]:
# def get_air_json(city_name, AIR_QUALITY_API_KEY):
#     return requests.get(f'https://api.waqi.info/feed/{city_name}/?token={AIR_QUALITY_API_KEY}').json()['data']

# Backfill Features

## Get Archived Data

In [ ]:
# periods = ["2015H1", "2016H1", "2017H1", "2018H1", "2019Q1", "2019Q2", "2019Q3", "2019Q4", "2020Q1", "2020Q2", "2020Q3", "2020Q4", "2021Q1", "2021Q2", "2021Q3", "2021Q4", "2022Q1", "2022Q2", "2022Q3", "2022Q4", "2023Q1", "2023Q2", "2023Q3", "2023Q4"]
# url = f"https://aqicn.org/data-platform/covid19/report/45376-1cdf91e1/{period}"

hist_url = f"aqicn.org/historical/#city:poland/slaski/katowice--ul-kossutha-6"

In [96]:
from io import StringIO
def get_archived_data(period, cities):
    url = f"https://aqicn.org/data-platform/covid19/report/45376-1cdf91e1/{period}"
    response = requests.get(url)
    if response.status_code == 200:
        # Read the CSV data into a DataFrame
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, delimiter=',', skiprows=4)
        # Filter the DataFrame to include only the rows with cities in the list
        filtered_df = df[df['City'].isin(cities)]
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
    return filtered_df

In [ ]:
archived_df = pd.DataFrame()
for period in periods:
    period_df = get_archived_data(period, cities)
    archived_df = pd.concat([archived_df, period_df], ignore_index=True)
    print(f"Added {len(period_df)} rows from period {period}")
print(archived_df.head())

Added 7427 rows from period 2015H1
Added 7700 rows from period 2016H1
Added 8306 rows from period 2017H1
Added 11035 rows from period 2018H1
Added 9493 rows from period 2019Q1
Added 9776 rows from period 2019Q2
Added 10384 rows from period 2019Q3
Added 10469 rows from period 2019Q4
Added 9429 rows from period 2020Q1
Added 10685 rows from period 2020Q2
Added 10697 rows from period 2020Q3
Added 10714 rows from period 2020Q4
Added 10745 rows from period 2021Q1
Added 10887 rows from period 2021Q2
Added 11042 rows from period 2021Q3
Added 10691 rows from period 2021Q4


C:\Users\sadai\AppData\Local\Temp\ipykernel_1920\3864398301.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, period_df], ignore_index=True)


Added 0 rows from period 2022Q1
Added 10795 rows from period 2022Q2
Added 10842 rows from period 2022Q3
Added 11093 rows from period 2022Q4
Added 11118 rows from period 2023Q1
Added 11174 rows from period 2023Q2
Added 11369 rows from period 2023Q3
Added 0 rows from period 2023Q4
         Date Country    City Specie count   min   max  median  variance
0  2015-05-18      US  Boston     o3   112  17.6  40.8    33.6    186.90
1  2015-05-21      US  Boston     o3   117   4.0  40.8    30.4    897.62
2  2015-01-09      US  Boston     o3    89   4.8  24.1    16.0    209.27
3  2015-02-15      US  Boston     o3   113  14.5  31.2    27.2    138.58
4  2015-02-16      US  Boston     o3   105  16.0  32.8    27.2    202.06


### Format Data

In [109]:
full_df.head()

,Date,Country,City,Specie,count,min,max,median,variance
0,2015-05-18,US,Boston,o3,112,17.6,40.8,33.6,186.90
1,2015-05-21,US,Boston,o3,117,4.0,40.8,30.4,897.62
2,2015-01-09,US,Boston,o3,89,4.8,24.1,16.0,209.27
3,2015-02-15,US,Boston,o3,113,14.5,31.2,27.2,138.58
4,2015-02-16,US,Boston,o3,105,16.0,32.8,27.2,202.06


In [107]:
# Pivot the DataFrame to get the desired format
pivot_df = full_df.pivot_table(index='Date', columns='Specie', values='median', aggfunc='first').reset_index()


In [108]:
pivot_df.head()

Specie,Date,co,dew,humidity,no2,o3,pm10,pm25,precipitation,pressure,so2,temperature,wd,wind gust,wind speed,wind-gust,wind-speed
0,2014-12-29,6.7,NaN,NaN,8.8,19.2,21.0,15.0,NaN,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-12-30,6.7,NaN,NaN,6.0,20.8,20.0,21.0,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-12-31,5.6,NaN,NaN,4.7,17.6,20.0,31.0,NaN,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,5.6,NaN,NaN,6.4,19.2,15.0,41.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-02,6.7,NaN,NaN,6.6,20.0,15.0,37.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
for city in cities:
    print(get_air_json(city, AQOPEN_API_KEY), '\n')

{'aqi': 31, 'idx': 5573, 'attributions': [{'url': 'https://soramame.env.go.jp/', 'name': 'Japan Atmospheric Environmental Regional Observation System (環境省大気汚染物質広域監視システム)', 'logo': 'Japan-Soramame.png'}, {'url': 'https://www.kankyo.metro.tokyo.lg.jp/', 'name': 'Tokyo, Japan Environment Agency (東京都環境局)', 'logo': 'Japan-Tokyo.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [35.6414629, 139.6981712], 'name': 'Meguro (目黒)', 'url': 'https://aqicn.org/city/meguro', 'location': ''}, 'dominentpol': 'o3', 'iaqi': {'co': {'v': 4.5}, 'h': {'v': 28.2}, 'no2': {'v': 28.8}, 'o3': {'v': 31.2}, 'p': {'v': 1018.3}, 'pm10': {'v': 14}, 'pm25': {'v': 1}, 'r': {'v': 99}, 'so2': {'v': 1.5}, 't': {'v': 9.9}, 'w': {'v': 3}}, 'time': {'s': '2025-03-14 08:00:00', 'tz': '+09:00', 'v': 1741939200, 'iso': '2025-03-14T08:00:00+09:00'}, 'forecast': {'daily': {'o3': [{'avg': 3, 'day': '2025-03-12', 'max': 10, 'min': 1}, {'avg': 4, 'day': '2025-03-13', 'max': 10, 'min':

In [17]:
r = requests.get(f"http://api.waqi.info/feed/{cities[4]}/?token={AQOPEN_API_KEY}")

In [18]:
r.json()['data']['iaqi']

{'co': {'v': 3.6},
 'h': {'v': 57},
 'no2': {'v': 11.5},
 'o3': {'v': 24.3},
 'p': {'v': 1013.6},
 'pm10': {'v': 12},
 'pm25': {'v': 30},
 'so2': {'v': 0.7},
 't': {'v': 7.6},
 'w': {'v': 4.3}}

In [90]:
r = requests.get(f"http://api.waqi.info//feed/geo:32.4069;-86.2564/?token={AQOPEN_API_KEY}")

In [ ]:
df = json_normalize(r.json()['data']['forecast']['daily'])   
df


# Get Today's Features